In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
from torch import nn 

# Teaching a quadruped to walk  

Time to try out the learning algorithms that you just implemented on a more difficult problem. The WalkerEnv implements a quadruped robot kind-of thing, see for yourself. The goal is to move in the $x$ direction as fast and as far as possible. 

Your goal is to implement a class ``WalkerPolicy`` with function ``determine_actions()`` just like the StochasticPolicy we used earlier to control the pendulum. Below is a template of this class, but feel free to alter it however you want. The only important thing is the ``determine_actions()`` function! 

After you implement it, copy ``WalkerPolicy`` into a separate file ``WalkerPolicy.py`` that you will upload to BRUTE together with the (optional) learned weights in a zip file. How the policy is implemented is up to you! You are constrained to only the libraries we used so far though, such as torch, numpy etc..

You will get some free points just for uploading a working policy (irrelevant of the performance). Further 2 points will be awarded for successfully traversing a small distance in the x direction.

Do mind, that the evaluation system might uses ``torch==1.12.1``.

# Hints

There is no single easy way of doing this, but here are some suggestions on what you could try to improve your policy:

1) This problem is much more difficult, than balancing a pendulum. It is a good idea to use a bit larger network than for the pendulum policy.

2) You can also try to use a different optimizer, such as Adam and play with the hyperparameters.

3) Using a neural network to compute the normal distribution scale $\sigma$ can lead to too much randomness in the actions (i.e. exploration). You can use a fixed $\sigma$ instead, or replace it with a learnable ```torch.Parameter``` initialized to some small constant. Make sure, you run it through an exponential, or softplus function to ensure $\sigma$ is positive.

4) The exploration can also be reduced by penalizing the variance of the action distribution in an additional loss term. 

5) If you see some undesirable behaviour, you can tweak the reward function to penalize it. Even though the $x$ distance is all we care about, adding extra terms to the reward can help guide the learning process (This is known as reward shaping). Simply define a reward function mapping the state $s_{t+1}$ and action $a_t$ to a scalar reward $r_t$ and put it in the config dictionary under the key ```'reward_fcn'```. See the ```WalkerEnv``` class for the implementation of the default reward.

6) Using the normal distribution on a bounded action space can lead to certain problems caused by action clipping. This can be mitigated by using a different distribution, such as the Beta distribution. See the ```torch.distributions.beta``` module for more information. (Note that Beta distribution is defined on the interval [0,1] and works better with parameters $\alpha,\beta \geq 1$.)

In [1]:
from environment.WalkerEnv import WalkerEnv

In [4]:
class WalkerPolicy(nn.Module):
    def __init__(self, state_dim=29, action_dim=8):
        super().__init__()
        # self.load_weights()  # load learned stored network weights after initialization

    # TODO: implement a determine_actions() function mapping from (N, state_dim) states into (N, action_dim) actions

    def save_weights(self, path='walker_weights.pt'):
        # helper function to save your network weights
        torch.save(self.state_dict(), path)

    def load_weights(self, path='walker_weights.pt'):
        # helper function to load your network weights
        self.load_state_dict(torch.load(path))

In [5]:
config = {'N': 1, 'vis': 1}
env = WalkerEnv(config)

Environment ready


In [13]:
obs = env.vector_reset()
env.render()

In [ ]:
obs[0, 0]  # this is the x coordinate of the robot, we want to maximize this

In [16]:
for i in range(512):
    a = np.random.randn(1, 8)
    obs, reward = env.vector_step(a)
env.close()